In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [13]:
url_inicial = 'http://culturales.uabc.mx/index.php/Culturales/issue/archive/2'
url_root = 'http://culturales.uabc.mx/index.php/Culturales/issue/archive/2'
r=requests.get(url_inicial)

In [14]:
soup = BeautifulSoup(r.text, 'html.parser')

In [15]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://culturales.uabc.mx/index.php/Culturales/issue/view/12',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/14',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/10',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/9',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/8',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/7',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/6',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/5',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/4',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/3',
 'http://culturales.uabc.mx/index.php/Culturales/issue/view/2']

In [16]:
def get_url_items(sopa, url): 
    box = soup.find('ul', class_='issues_archive')
    volumen=box.findAll('div', class_='obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [17]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina http://culturales.uabc.mx/index.php/Culturales/issue/archive/2


In [18]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

11

In [19]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [20]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Culturales'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Estudios Culturales'
    url='http://culturales.uabc.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h3', class_='title').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [21]:
list_scraper=list_scraper[0:20]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10


In [22]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,"No soy mexicano, soy de Tijuana: juventud e id...",,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
1,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,Consumo de bienes culturales: reflexiones sobr...,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
2,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,"Más allá de la sociología: Sobre catástrofes, ...",,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
3,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,El suspenso del melodrama: Entre modelos narra...,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
4,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,Identidades sociales en jóvenes de sectores po...,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
5,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,La subsunción simbólica,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
6,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,"El desierto como territorio, paisaje y referen...",,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
7,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,Nosotros y los otros. Una aproximación reflexiva,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
8,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,La “gente del desierto” en el norte de Sonora,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...
9,Culturales,IV. Humanidades y Ciencias de la Conducta,Estudios Culturales,La inscripción de la ley en los cuerpos: un re...,,http://culturales.uabc.mx,http://culturales.uabc.mx/index.php/Culturales...


In [23]:
df_catalogo.to_csv('Revista175.02.csv', index=False)